In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import psycopg2
import json
import random
import string

def connectDb_func(DbConfig):
    conn = psycopg2.connect(**DbConfig)
    conn.autocommit = True
    return conn

def insert_func(DbConfig,data,tableName):
    data = data.fillna('NULL')
    sql = '''INSERT INTO {} ({}) VALUES {};'''.format(
        tableName
        , ",".join(['"{}"'.format(c) for c in data.columns])
        , ','.join(['('+",".join(["'{}'".format(v)  if type(v)==str else '{}'.format(v) for v in row])+')' for row in data.values])
    )
    conn = connectDb_func(DbConfig)
    cursor = conn.cursor()
    
    cursor.execute(sql)
    conn.commit()
    cursor.close()
    conn.close()
    return

def query_func(DbConfig,sql):
    conn = connectDb_func(DbConfig)
    cursor = conn.cursor()
    
    cursor.execute(sql)
    data = cursor.fetchall()
    colnames = [desc[0] for desc in cursor.description]
    data = {
        'data' : data
        , 'columns' : colnames
    }
    
    cursor.close()
    conn.close()
    return data

In [ ]:
### load config connection to database
DbConfig = json.load(open('config.json'))

In [2]:
%%time
### load and preparing
dict_dtype = {
    'Amount Requested' : 'float64'
    , 'Application Date' : 'str'
    , 'Loan Title' : 'str'
    , 'Risk_Score' : 'float64'
    , 'Debt-To-Income Ratio' : 'str'
    , 'Zip Code' : 'str'
    , 'State' : 'str'
    , 'Employment Length' : 'str'
    , 'Policy Code' : 'float64'
}

df = pd.read_csv('rejected_2007_to_2018Q4.csv',dtype = dict_dtype)
df['Debt-To-Income Ratio'] = df['Debt-To-Income Ratio'].str.replace('%','').astype('float64')
df['Loan Title'] = df['Loan Title'].str.replace("'","")
df = df.sort_values('Application Date').reset_index(drop=True)
df['Risk_Score'] = df['Risk_Score'].fillna(0)

CPU times: user 48 s, sys: 5.94 s, total: 53.9 s
Wall time: 54.1 s


In [3]:
%%time
# resmapling and save real data
df[df['Application Date'] <= '2007-12-31'].head(1000).to_csv('rejected2007.csv',index=False)
df[df['Application Date'] > '2008-12-31'].head(1000).to_csv('rejected2009.csv',index=False)

In [199]:
%%time
## mock data 
list_date = [dt.datetime(2008,12,31) - dt.timedelta(days=x) for x in range(366)]
text = list(string.ascii_lowercase)
Employ = list(df['Employment Length'].unique())[:-2]
pd.DataFrame(
    {
        'Amount Requested' : random.sample(range(0, 10000), len(list_date))
        , 'Application Date' : list_date
        , 'Loan Title' : ['mock_{}'.format(x) for x in range(len(list_date))]
        , 'Risk_Score' : random.sample(range(0, 500), len(list_date))
        , 'Debt-To-Income Ratio' : random.sample(range(0, 500), len(list_date))
        , 'Zip Code' : ['{}xx'.format(x) for x in random.sample(range(100, 1000), len(list_date))]
        , 'State' : [''.join(random.choices(text, k=2)) for x in range(len(list_date))]
        , 'Employment Length' : [random.choices(Employ, k=1)[0] for x in range(len(list_date))]
        , 'Policy Code' : [random.choices([1,2], k=1)[0] for x in range(len(list_date))]
    }
).sort_values('Application Date').to_csv('rejected2008.csv',index=False)